In [85]:
from bs4 import BeautifulSoup
# import pandas as pd
from datetime import datetime, timedelta
from itertools import combinations
import json
import requests
import math
from typing import Optional, List, Dict

from pydantic import BaseModel
from typing import Mapping
import numpy as np
import polars as pl
import hvplot.polars
import pandas as pd
import altair as alt
import math

from shared.utils import formalize_data_with_str
from shared.random_rules import RandomRules

In [86]:
rr = RandomRules(
  TOTAL_NUMBERS = 35,
  CHOOSE = 5,
  MAX_PRIMES = 3,
  N_VALUES=["1", "2", "3"],
  # Rangos de valores de las bolillas
  MIN_B_VALUES = [1, 2, 3, 4, 5],
  MAX_B_VALUES = [31, 32, 33, 34, 35]
)

In [87]:
url = 'https://resultados.latinka.com.pe/i.do?m=historico&t=0&s=164'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

tables = soup.select('table')
rows = tables[1].find_all('tr')

In [88]:
lst = []
post_str_b1 = None
post_str_blast = None
for row in rows:
  cols = [data.text.strip() for data in row.find_all('td')]
  
  id, bolillas_int, str_combo = formalize_data_with_str(bolillas_str=cols[2])
  cols.append(id)
  cols.append(bolillas_int)
  cols.extend([b for b in bolillas_int])
  cols.extend(str_combo)
  cols.append(post_str_b1)
  cols.append(post_str_blast)

  is_rare, reason = rr.is_rare_combination(bolillas_int, id)
  cols.append(is_rare)
  cols.append(reason)

  cols.append(rr.jumps_map(bolillas_int))
  cols.append(rr.unijump(bolillas_int))
  
  lst.append(cols)

  post_str_b1 = str_combo[0]
  post_str_blast = str_combo[-1]

with open('./gn_cahe_v3.json', 'w') as file: 
  json.dump(lst, file)

dfRows = pl.DataFrame(
  lst,
  schema=[
    'fecha', 'sorteo', 'bolillas',
    'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5',
    'str_b1', 'str_b2', 'str_b3', 'str_b4', 'str_b5',
    'post_str_b1', 'post_str_blast',
    'is_rare_combination', 'reason',
    "jumps_map", "unijump"
  ],
  schema_overrides={
    'combo': pl.Array(pl.UInt8, 5),
    'b1': pl.UInt8,
    'b2': pl.UInt8,
    'b3': pl.UInt8,
    'b4': pl.UInt8,
    'b5': pl.UInt8,
    'b6': pl.UInt8,
    'reason': pl.UInt8
  },
  orient='row'
)

TOTAL_LINES = len(dfRows)
print(f'{TOTAL_LINES=}')
display(dfRows.head(10))

TOTAL_LINES=7718


fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""02/04/2025""","""4174""","""26 14 24 11 17""","""1114172426""","[11, 14, … 26]",11,14,17,24,26,"""11""","""14""","""17""","""24""","""26""",null,null,true,24,"""03_03_07_02""",-13
"""01/04/2025""","""4173""","""12 25 33 13 08""","""0812132533""","[8, 12, … 33]",8,12,13,25,33,"""08""","""12""","""13""","""25""","""33""","""11""","""26""",false,0,"""04_01_12_08""",-29
"""31/03/2025""","""4172""","""07 22 34 26 16""","""0716222634""","[7, 16, … 34]",7,16,22,26,34,"""07""","""16""","""22""","""26""","""34""","""08""","""33""",false,0,"""09_06_04_08""",5
"""30/03/2025""","""4171""","""35 05 15 29 03""","""0305152935""","[3, 5, … 35]",3,5,15,29,35,"""03""","""05""","""15""","""29""","""35""","""07""","""34""",true,2,"""02_10_14_06""",-8
"""29/03/2025""","""4170""","""23 06 04 09 10""","""0406091023""","[4, 6, … 23]",4,6,9,10,23,"""04""","""06""","""09""","""10""","""23""","""03""","""35""",false,0,"""02_03_01_13""",17
"""28/03/2025""","""4169""","""20 10 01 09 29""","""0109102029""","[1, 9, … 29]",1,9,10,20,29,"""01""","""09""","""10""","""20""","""29""","""04""","""23""",true,16,"""08_01_10_09""",-26
"""27/03/2025""","""4168""","""27 35 05 32 08""","""0508273235""","[5, 8, … 35]",5,8,27,32,35,"""05""","""08""","""27""","""32""","""35""","""01""","""29""",true,16,"""03_19_05_03""",42
"""26/03/2025""","""4167""","""34 29 06 24 26""","""0624262934""","[6, 24, … 34]",6,24,26,29,34,"""06""","""24""","""26""","""29""","""34""","""05""","""35""",true,16,"""18_02_03_05""",-16
"""25/03/2025""","""4166""","""29 06 35 22 04""","""0406222935""","[4, 6, … 35]",4,6,22,29,35,"""04""","""06""","""22""","""29""","""35""","""06""","""34""",true,16,"""02_16_07_06""",31


In [89]:
import holoviews as hv
def plot_post_col (col: str, post_col: str, test_value: str):
  # col = 'str_b1'
  # post_col = 'post_str_b1'
  df_viz = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      post_col
    ).agg(pl.count(post_col).alias('count')) \
    .sort("count", descending=True)
  
  # Agrega una columna que sea el porcentaje de la frecuencia relativa
  total = df_viz['count'].sum()
  df_viz = df_viz.with_columns(
    ( (pl.col('count') / total) * 100 ).alias('percent')
  )
  mean_percent = df_viz['percent'].mean()
  print(mean_percent)

  chart = df_viz.hvplot.bar(
    x=post_col,
    y='percent'
  )

  chart = chart * hv.HLine(mean_percent).opts(color='red')
  chart.opts(default_tools=["pan"])


  display(df_viz)
  display(chart)

In [90]:
plot_post_col(col='str_b1', post_col='post_str_b1', test_value="11")

4.545454545454544


post_str_b1,count,percent
str,u32,f64
"""02""",48,16.551724
"""01""",38,13.103448
"""03""",37,12.758621
"""07""",23,7.931034
"""04""",23,7.931034
…,…,…
"""21""",2,0.689655
"""18""",1,0.344828
"""23""",1,0.344828


:Overlay
   .Bars.I  :Bars   [post_str_b1]   (percent)
   .HLine.I :HLine   [x,y]

In [91]:
plot_post_col(col='str_b5', post_col='post_str_blast', test_value="26")

4.3478260869565215


post_str_blast,count,percent
str,u32,f64
"""33""",44,13.017751
"""35""",42,12.426036
"""32""",36,10.650888
"""34""",35,10.35503
"""30""",26,7.692308
…,…,…
"""18""",1,0.295858
"""13""",1,0.295858
"""12""",1,0.295858


:Overlay
   .Bars.I  :Bars   [post_str_blast]   (percent)
   .HLine.I :HLine   [x,y]